In [1]:
!unzip -j /content/utils.zip -d.

Archive:  /content/utils.zip
  inflating: ./weather_classification_data.csv  
  inflating: ./data_scaled_labeled_check.csv  


In [3]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


In [5]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
2775,33.0,72,12.0,37.0,overcast,1000.20,2,Spring,8.0,coastal,Cloudy
5946,2.0,89,1.5,69.0,overcast,994.98,0,Winter,3.5,mountain,Snowy
8065,27.0,65,13.0,83.0,partly cloudy,1002.82,0,Winter,5.0,coastal,Rainy
1206,76.0,82,14.0,106.0,clear,1023.29,12,Winter,8.0,coastal,Sunny
172,30.0,58,1.0,43.0,overcast,1018.95,2,Winter,6.0,inland,Cloudy


In [6]:
# START CODE HERE
object_columns = data.select_dtypes(include=['object']).columns.tolist()
non_object_columns = data.select_dtypes(exclude=['object']).columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")


Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


In [7]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")


Test Passed


In [8]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

# Creating the ColumnTransformer
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), object_columns),
        ('num', StandardScaler(), non_object_columns)
    ],
    remainder='passthrough'
)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)


In [9]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


In [10]:
from sklearn.model_selection import train_test_split

# Drop the Weather Type column to get X and pick the Weather Type column to get y
X = data_scaled_labeled.drop(columns=["Weather Type"])
y = data_scaled_labeled["Weather Type"]

# 90-10 split of data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=10)

In [11]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [12]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,3.0,2.0,1.0,0.970479,-1.718913,-1.061339,-1.241007,0.172646,1.554340,1.049153
12278,2.0,3.0,1.0,-1.330267,-0.183764,-0.265211,0.073739,-0.631426,-1.038715,-1.027154
2859,2.0,3.0,2.0,-1.617860,0.261924,0.024290,0.887629,-0.505076,-0.779410,-0.730539
5495,3.0,0.0,0.0,0.797923,0.410487,-0.699463,-0.207993,0.011885,-0.260799,-0.137308
8469,2.0,1.0,2.0,0.567848,0.410487,-0.844213,-0.458420,0.047102,-0.520104,0.604230


In [13]:
# START CODE HERE

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=10)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)

# END CODE HERE

print("Accuracy:", score * 100)


Accuracy: 91.13636363636364


In [14]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


In [15]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,2.0,3.0,1.0,-1.157711,-0.282806,0.530916,1.169360,-0.230869,-0.779410,-0.137308
2331,3.0,3.0,1.0,0.567848,1.945636,2.340296,1.263271,0.205712,1.813646,-0.285616
217,2.0,1.0,2.0,-0.007338,1.301864,1.109918,-0.020172,-0.123068,-0.520104,-0.730539
1760,3.0,3.0,0.0,2.063333,0.509529,-0.265211,1.200664,0.152752,1.554340,-1.323769
3963,3.0,0.0,2.0,0.165218,-2.164601,-0.192836,-1.334918,0.258671,1.554340,0.604230


In [16]:
y = model.predict(input)
y

array([2., 0., 1., 0., 3.])